In [1]:
from widerface_evaluate.evaluation import get_preds

In [2]:
preds = get_preds('./widerface_evaluate/widerface_txt/')

Reading Predictions : 100%|██████████| 61/61 [00:00<00:00, 77.65it/s] 


In [3]:
type(preds)

dict

In [4]:
list(preds.keys())

['27--Spa',
 '1--Handshaking',
 '55--Sports_Coach_Trainer',
 '35--Basketball',
 '38--Tennis',
 '24--Soldier_Firing',
 '53--Raid',
 '49--Greeting',
 '32--Worker_Laborer',
 '4--Dancing',
 '31--Waiter_Waitress',
 '47--Matador_Bullfighter',
 '19--Couple',
 '28--Sports_Fan',
 '61--Street_Battle',
 '40--Gymnastics',
 '2--Demonstration',
 '42--Car_Racing',
 '58--Hockey',
 '14--Traffic',
 '26--Soldier_Drilling',
 '12--Group',
 '37--Soccer',
 '50--Celebration_Or_Party',
 '43--Row_Boat',
 '22--Picnic',
 '11--Meeting',
 '59--people--driving--car',
 '51--Dresses',
 '45--Balloonist',
 '10--People_Marching',
 '25--Soldier_Patrol',
 '13--Interview',
 '41--Swimming',
 '5--Car_Accident',
 '17--Ceremony',
 '9--Press_Conference',
 '44--Aerobics',
 '21--Festival',
 '23--Shoppers',
 '0--Parade',
 '33--Running',
 '57--Angler',
 '6--Funeral',
 '30--Surgeons',
 '46--Jockey',
 '3--Riot',
 '29--Students_Schoolkids',
 '8--Election_Campain',
 '15--Stock_Market',
 '34--Baseball',
 '20--Family_Group',
 '18--Concert

In [5]:
type(preds['16--Award_Ceremony'])

dict

In [6]:
list(preds['16--Award_Ceremony'].keys())

['16_Award_Ceremony_Awards_Ceremony_16_226',
 '16_Award_Ceremony_Awards_Ceremony_16_305',
 '16_Award_Ceremony_Awards_Ceremony_16_589',
 '16_Award_Ceremony_Awards_Ceremony_16_392',
 '16_Award_Ceremony_Awards_Ceremony_16_56',
 '16_Award_Ceremony_Awards_Ceremony_16_750',
 '16_Award_Ceremony_Awards_Ceremony_16_73',
 '16_Award_Ceremony_Awards_Ceremony_16_512',
 '16_Award_Ceremony_Awards_Ceremony_16_116',
 '16_Award_Ceremony_Awards_Ceremony_16_447',
 '16_Award_Ceremony_Awards_Ceremony_16_569',
 '16_Award_Ceremony_Awards_Ceremony_16_752',
 '16_Award_Ceremony_Awards_Ceremony_16_84',
 '16_Award_Ceremony_Awards_Ceremony_16_231',
 '16_Award_Ceremony_Awards_Ceremony_16_361',
 '16_Award_Ceremony_Awards_Ceremony_16_467',
 '16_Award_Ceremony_Awards_Ceremony_16_474',
 '16_Award_Ceremony_Awards_Ceremony_16_546',
 '16_Award_Ceremony_Awards_Ceremony_16_495',
 '16_Award_Ceremony_Awards_Ceremony_16_195',
 '16_Award_Ceremony_Awards_Ceremony_16_85',
 '16_Award_Ceremony_Awards_Ceremony_16_309',
 '16_Award_Cer

In [7]:
preds['16--Award_Ceremony']['16_Award_Ceremony_Awards_Ceremony_16_25']

array([[ 5.02000000e+02,  9.40000000e+01,  3.30000000e+01,
         4.40000000e+01,  9.98788900e-01],
       [ 4.40000000e+02,  1.22000000e+02,  3.30000000e+01,
         4.30000000e+01,  9.97949060e-01],
       [ 8.43000000e+02,  1.33000000e+02,  3.80000000e+01,
         4.70000000e+01,  9.97211640e-01],
       [ 6.50000000e+02,  1.37000000e+02,  3.80000000e+01,
         4.70000000e+01,  9.96059950e-01],
       [ 5.63000000e+02,  1.03000000e+02,  4.00000000e+01,
         5.40000000e+01,  9.95707300e-01],
       [ 2.25000000e+02,  1.18000000e+02,  3.90000000e+01,
         4.90000000e+01,  9.92876200e-01],
       [ 7.40000000e+02,  1.46000000e+02,  4.20000000e+01,
         5.50000000e+01,  9.90303930e-01],
       [ 9.29000000e+02,  1.53000000e+02,  4.20000000e+01,
         5.40000000e+01,  9.86212000e-01],
       [ 1.49000000e+02,  9.60000000e+01,  3.90000000e+01,
         5.40000000e+01,  9.85900000e-01],
       [ 3.32000000e+02,  1.48000000e+02,  4.30000000e+01,
         5.20000000e+01

In [8]:
preds['16--Award_Ceremony']['16_Award_Ceremony_Awards_Ceremony_16_25'].shape

(52, 5)

# Generate predictions

In [17]:
from __future__ import print_function
import os
import argparse
import torch
import torch.backends.cudnn as cudnn
import numpy as np
from data import cfg_mnet, cfg_re50
from layers.functions.prior_box import PriorBox
from utils.nms.py_cpu_nms import py_cpu_nms
import cv2
from models.retinaface import RetinaFace
from utils.box_utils import decode, decode_landm
from utils.timer import Timer


parser = argparse.ArgumentParser(description='Retinaface')
parser.add_argument('-m', '--trained_model', default='./weights/Resnet50_Final.pth',
                    type=str, help='Trained state_dict file path to open')
parser.add_argument('--network', default='resnet50', help='Backbone network mobile0.25 or resnet50')
parser.add_argument('--origin_size', default=True, type=str, help='Whether use origin image size to evaluate')
parser.add_argument('--save_folder', default='./widerface_evaluate/widerface_txt/', type=str, help='Dir to save txt results')
parser.add_argument('--cpu', action="store_true", default=False, help='Use cpu inference')
parser.add_argument('--dataset_folder', default='./data/widerface/val/images/', type=str, help='dataset path')
parser.add_argument('--confidence_threshold', default=0.02, type=float, help='confidence_threshold')
parser.add_argument('--top_k', default=5000, type=int, help='top_k')
parser.add_argument('--nms_threshold', default=0.4, type=float, help='nms_threshold')
parser.add_argument('--keep_top_k', default=750, type=int, help='keep_top_k')
parser.add_argument('-s', '--save_image', action="store_true", default=False, help='show detection results')
parser.add_argument('--vis_thres', default=0.5, type=float, help='visualization_threshold')
#args = parser.parse_args()

args = argparse.Namespace()
args.trained_model = './weights/pretrain/mobilenet0.25_Final.pth'
args.network = 'mobile0.25'
args.origin_size = True
args.save_folder = './widerface_evaluate/widerface_txt/'
args.cpu = False
args.dataset_folder = './data/widerface/val/images/'
args.confidence_threshold = 0.02
args.top_k = 5000
args.nms_threshold = 0.4
args.keep_top_k = 750
args.save_image = False
args.vis_thres = 0.5


def check_keys(model, pretrained_state_dict):
    ckpt_keys = set(pretrained_state_dict.keys())
    model_keys = set(model.state_dict().keys())
    used_pretrained_keys = model_keys & ckpt_keys
    unused_pretrained_keys = ckpt_keys - model_keys
    missing_keys = model_keys - ckpt_keys
    print('Missing keys:{}'.format(len(missing_keys)))
    print('Unused checkpoint keys:{}'.format(len(unused_pretrained_keys)))
    print('Used keys:{}'.format(len(used_pretrained_keys)))
    assert len(used_pretrained_keys) > 0, 'load NONE from pretrained checkpoint'
    return True


def remove_prefix(state_dict, prefix):
    ''' Old style model is stored with all names of parameters sharing common prefix 'module.' '''
    print('remove prefix \'{}\''.format(prefix))
    f = lambda x: x.split(prefix, 1)[-1] if x.startswith(prefix) else x
    return {f(key): value for key, value in state_dict.items()}


def load_model(model, pretrained_path, load_to_cpu):
    print('Loading pretrained model from {}'.format(pretrained_path))
    if load_to_cpu:
        pretrained_dict = torch.load(pretrained_path, map_location=lambda storage, loc: storage)
    else:
        device = torch.cuda.current_device()
        pretrained_dict = torch.load(pretrained_path, map_location=lambda storage, loc: storage.cuda(device))
    if "state_dict" in pretrained_dict.keys():
        pretrained_dict = remove_prefix(pretrained_dict['state_dict'], 'module.')
    else:
        pretrained_dict = remove_prefix(pretrained_dict, 'module.')
    check_keys(model, pretrained_dict)
    model.load_state_dict(pretrained_dict, strict=False)
    return model




In [39]:
from collections import defaultdict
def predict():
    torch.set_grad_enabled(False)

    cfg = None
    if args.network == "mobile0.25":
        cfg = cfg_mnet
    elif args.network == "resnet50":
        cfg = cfg_re50
    # net and model
    net = RetinaFace(cfg=cfg, phase = 'test')
    net = load_model(net, args.trained_model, args.cpu)
    net.eval()
    print('Finished loading model!')
    #print(net)
    cudnn.benchmark = True
    device = torch.device("cpu" if args.cpu else "cuda")
    net = net.to(device)

    # testing dataset
    testset_folder = args.dataset_folder
    testset_list = args.dataset_folder[:-7] + "wider_val.txt"

    with open(testset_list, 'r') as fr:
        test_dataset = fr.read().split()
    num_images = len(test_dataset)

    _t = {'forward_pass': Timer(), 'misc': Timer()}

    # testing begin
    results = defaultdict(dict)
    for i, img_name in enumerate(test_dataset):
        image_path = testset_folder + img_name
        img_raw = cv2.imread(image_path, cv2.IMREAD_COLOR)
        img = np.float32(img_raw)

        # testing scale
        target_size = 1600
        max_size = 2150
        im_shape = img.shape
        im_size_min = np.min(im_shape[0:2])
        im_size_max = np.max(im_shape[0:2])
        resize = float(target_size) / float(im_size_min)
        # prevent bigger axis from being more than max_size:
        if np.round(resize * im_size_max) > max_size:
            resize = float(max_size) / float(im_size_max)
        if args.origin_size:
            resize = 1

        if resize != 1:
            img = cv2.resize(img, None, None, fx=resize, fy=resize, interpolation=cv2.INTER_LINEAR)
        im_height, im_width, _ = img.shape
        scale = torch.Tensor([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])
        img -= (104, 117, 123)
        img = img.transpose(2, 0, 1)
        img = torch.from_numpy(img).unsqueeze(0)
        img = img.to(device)
        scale = scale.to(device)

        _t['forward_pass'].tic()
        loc, conf, landms = net(img)  # forward pass
        _t['forward_pass'].toc()
        _t['misc'].tic()
        priorbox = PriorBox(cfg, image_size=(im_height, im_width))
        priors = priorbox.forward()
        priors = priors.to(device)
        prior_data = priors.data
        boxes = decode(loc.data.squeeze(0), prior_data, cfg['variance'])
        boxes = boxes * scale / resize
        boxes = boxes.cpu().numpy()
        scores = conf.squeeze(0).data.cpu().numpy()[:, 1]
        landms = decode_landm(landms.data.squeeze(0), prior_data, cfg['variance'])
        scale1 = torch.Tensor([img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                               img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                               img.shape[3], img.shape[2]])
        scale1 = scale1.to(device)
        landms = landms * scale1 / resize
        landms = landms.cpu().numpy()

        # ignore low scores
        inds = np.where(scores > args.confidence_threshold)[0]
        boxes = boxes[inds]
        landms = landms[inds]
        scores = scores[inds]

        # keep top-K before NMS
        order = scores.argsort()[::-1]
        # order = scores.argsort()[::-1][:args.top_k]
        boxes = boxes[order]
        landms = landms[order]
        scores = scores[order]

        # do NMS
        dets = np.hstack((boxes, scores[:, np.newaxis])).astype(np.float32, copy=False)
        keep = py_cpu_nms(dets, args.nms_threshold)
        # keep = nms(dets, args.nms_threshold,force_cpu=args.cpu)
        dets = dets[keep, :]
        landms = landms[keep]

        # keep top-K faster NMS
        # dets = dets[:args.keep_top_k, :]
        # landms = landms[:args.keep_top_k, :]

        dets = np.concatenate((dets, landms), axis=1)
        _t['misc'].toc()
        
        event = img_name.split('/')[1]
        event_img = img_name.split('/')[2][:-4]

        if event_img not in results[event]:
            results[event][event_img] = []
            
        # --------------------------------------------------------------------
        save_name = args.save_folder + img_name[:-4] + ".txt"
        dirname = os.path.dirname(save_name)
        if not os.path.isdir(dirname):
            os.makedirs(dirname)
        with open(save_name, "w") as fd:
            bboxs = dets
            file_name = os.path.basename(save_name)[:-4] + "\n"
            bboxs_num = str(len(bboxs)) + "\n"
            fd.write(file_name)
            fd.write(bboxs_num)
            print('dir:', dirname)
            print('img_name:', img_name)
            print('file:', file_name)
            print(bboxs_num)
            for box in bboxs:
                x = int(box[0])
                y = int(box[1])
                w = int(box[2]) - int(box[0])
                h = int(box[3]) - int(box[1])
                confidence = str(box[4])
                line = str(x) + " " + str(y) + " " + str(w) + " " + str(h) + " " + confidence + " \n"
                #fd.write(line)
                print(line)

        #print('im_detect: {:d}/{:d} forward_pass_time: {:.4f}s misc: {:.4f}s'.format(i + 1, num_images, _t['forward_pass'].average_time, _t['misc'].average_time))
        #print('line:', line)
        print('dets:', dets)
        break
        

In [35]:
predict()

Loading pretrained model from ./weights/pretrain/mobilenet0.25_Final.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:300
Finished loading model!
dir: ./widerface_evaluate/widerface_txt//24--Soldier_Firing
img_name: /24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_329.jpg
file: 24_Soldier_Firing_Soldier_Firing_24_329

6

884 194 123 142 0.81570655 

882 346 105 114 0.090121895 

594 293 176 245 0.06757448 

344 179 328 473 0.025667015 

668 396 29 33 0.025203438 

845 186 179 259 0.021932088 

dets: [[8.8433685e+02 1.9427991e+02 1.0076625e+03 3.3636084e+02 8.1570655e-01
  9.0050262e+02 2.4671545e+02 9.4849384e+02 2.4304390e+02 9.1522015e+02
  2.8052585e+02 9.2020209e+02 3.0874371e+02 9.5279761e+02 3.0545010e+02]
 [8.8296826e+02 3.4696515e+02 9.8761560e+02 4.6025662e+02 9.0121895e-02
  9.1701556e+02 3.8799265e+02 9.5402771e+02 3.8312177e+02 9.4254816e+02
  4.0152563e+02 9.2681958e+02 4.2465784e+02 9.5531165e+02 4.2107053e+02]
 [5.9468719e+02 2.9342944

In [33]:
!ls ./widerface_evaluate/widerface_txt//24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_329.txt

./widerface_evaluate/widerface_txt//24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_329.txt


In [36]:
'/24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_329.jpg'.split('/')

['', '24--Soldier_Firing', '24_Soldier_Firing_Soldier_Firing_24_329.jpg']

In [37]:
'/24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_329.jpg'.split('/')[2][:-4]

'24_Soldier_Firing_Soldier_Firing_24_329'

In [42]:
results = defaultdict(dict)
results

defaultdict(dict, {})

In [43]:
results['a']

{}

In [44]:
tmp = defaultdict(list)
results = defaultdict(tmp)

TypeError: first argument must be callable or None